In [25]:
import pandas as pd
import numpy as np
import os

### Read in the data

In [26]:
player = pd.read_csv('https://raw.githubusercontent.com/jchristo12/fantasy_football/master/data/predictor/player_offense.csv')

In [27]:
#rename seas.1 columns
player.rename(columns={'seas.1': 'exp'}, inplace=True)

In [28]:
#convert column types to appropriate data types
player = player.astype({'seas': 'category',
          'wk': 'category',
          'team': 'category',
          'pos1': 'category',
          'dv': 'category',
          'exp': 'category'})

In [29]:
#convert dob to datetime
#player['dob'] = pd.to_datetime(player['dob'])

### Cumulative Stats

#### Career stats

In [30]:
player_sorted = player.sort_values(by=['player', 'gid'], ascending=True)

In [31]:
player_sorted.head()

,pk,gid,seas,wk,player,team,pos1,nflid,pa,pc,...,dob,dv,forty,bench,vertical,broad,shuttle,cone,arm,hand
74708,3999&AA-0025,3999,2015,1,AA-0025,DET,RB,2552374,0,0,...,6/13/1993,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
74935,4010&AA-0025,4010,2015,2,AA-0025,DET,RB,2552374,0,0,...,6/13/1993,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
75490,4036&AA-0025,4036,2015,3,AA-0025,DET,RB,2552374,0,0,...,6/13/1993,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
75814,4052&AA-0025,4052,2015,4,AA-0025,DET,RB,2552374,0,0,...,6/13/1993,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
76021,4062&AA-0025,4062,2015,5,AA-0025,DET,RB,2552374,0,0,...,6/13/1993,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625


In [32]:
player_sorted.loc[:, 'pa':'tdrec'].cumsum().head()

,pa,pc,py,ints,tdp,ra,sra,ry,tdr,fuml,trg,rec,recy,tdrec
74708,0,0,0,0,0,7,4,50,1,0,4,4,44,0
74935,0,0,0,0,0,13,7,59,1,0,5,5,53,0
75490,0,0,0,0,0,21,12,82,1,0,10,7,72,1
75814,0,0,0,0,0,34,17,115,1,0,15,9,83,1
76021,0,0,0,0,0,40,20,131,1,1,17,10,92,1


In [33]:
#initialize a list to hold all cumulative stats
cusum_stats = []

In [34]:
#loop through all unique players and calculate cumulative stats
for i in player_sorted['player'].unique():
    x = player_sorted[player_sorted['player'] == i].loc[:, 'pa':'tdrec'].cumsum()
    cusum_stats.append(x)

In [35]:
#combine all of the items in the list into 1 dataframe
cusum_df = pd.concat(cusum_stats, axis=0)

In [36]:
#check to see if length matches the original dataframe
print(cusum_df.shape[0])
print(player.shape[0])

94759
94759


Proof that this method is calculating correctly (lengths match)

In [37]:
#rename columns to add a cusum prefix
cusum_df = cusum_df.add_prefix('cusum_')

In [38]:
#add the cusum data to the sorted data frame
player_sorted_cusum = pd.concat([player_sorted, cusum_df], axis=1)

#### 4-game rolling

In [39]:
#initialize a list to hold each players rolling sum
recent_stats = []

In [40]:
#loop thru each unique player calculating rolling sum
for i in player_sorted['player'].unique():
    x = player_sorted[player_sorted['player'] == i].loc[:, 'pa':'tdrec'].rolling(window=4, min_periods=1).sum()
    recent_stats.append(x)

In [41]:
#combine the items in the recent stats list
recent_df = pd.concat(recent_stats, axis=0)

In [42]:
#make sure lengths match the original
print(recent_df.shape[0])
print(player.shape[0])

94759
94759


In [46]:
#rename columns to add a recent prefix
recent_df = recent_df.add_prefix('recent_')

In [47]:
#add the recent stats columns to the sorted data frame
player_rolling_sort = pd.concat([player_sorted_cusum, recent_df], axis=1)

### Add other features

In [49]:
player_rolling_sort.columns

Index(['pk', 'gid', 'seas', 'wk', 'player', 'team', 'pos1', 'nflid', 'pa',
       'pc', 'py', 'ints', 'tdp', 'ra', 'sra', 'ry', 'tdr', 'fuml', 'trg',
       'rec', 'recy', 'tdrec', 'exp', 'height', 'weight', 'dob', 'dv', 'forty',
       'bench', 'vertical', 'broad', 'shuttle', 'cone', 'arm', 'hand',
       'cusum_pa', 'cusum_pc', 'cusum_py', 'cusum_ints', 'cusum_tdp',
       'cusum_ra', 'cusum_sra', 'cusum_ry', 'cusum_tdr', 'cusum_fuml',
       'cusum_trg', 'cusum_rec', 'cusum_recy', 'cusum_tdrec', 'recent_pa',
       'recent_pc', 'recent_py', 'recent_ints', 'recent_tdp', 'recent_ra',
       'recent_sra', 'recent_ry', 'recent_tdr', 'recent_fuml', 'recent_trg',
       'recent_rec', 'recent_recy', 'recent_tdrec'],
      dtype='object')

#### Age

In [51]:
#strip out the year from DOB
dob_year = player_rolling_sort['dob'].apply(lambda x: int(str(x[-4:])))

In [52]:
dob_year.head()

74708    1993
74935    1993
75490    1993
75814    1993
76021    1993
Name: dob, dtype: int64

In [53]:
#add age to dataframe
player_rolling_sort['age'] = player_rolling_sort['seas'].astype(int) - dob_year

In [54]:
player_rolling_sort['age'].head()

74708    22
74935    22
75490    22
75814    22
76021    22
Name: age, dtype: int64

### Passing summaries

#### Career stats

In [55]:
#Passing completion
player_rolling_sort['career_comp_pct'] = player_rolling_sort['cusum_pc'] / player_rolling_sort['cusum_pa']

In [56]:
#touchdown to interception ratio
player_rolling_sort['career_td_to_int'] = player_rolling_sort['cusum_tdp'] / player_rolling_sort['cusum_ints']

In [57]:
#Passing yards per completion
player_rolling_sort['career_yds_per_comp'] = player_rolling_sort['cusum_py'] / player_rolling_sort['cusum_pc']

#### Recent stats

In [59]:
#Passing completion
player_rolling_sort['recent_comp_pct'] = player_rolling_sort['recent_pc'] / player_rolling_sort['recent_pa']

In [60]:
#touchdown to interception ratio
player_rolling_sort['recent_td_to_int'] = player_rolling_sort['recent_tdp'] / player_rolling_sort['recent_ints']

In [61]:
#Passing yards per completion
player_rolling_sort['recent_yds_per_comp'] = player_rolling_sort['recent_py'] / player_rolling_sort['recent_pc']

### Running summaries

#### Career stats

In [192]:
#Yards per rush
player_rolling_sort['career_ryds_per_carry'] = player_rolling_sort['cusum_ry'] / player_rolling_sort['cusum_ra']

In [195]:
#Carries to touchdown ratio
player_rolling_sort['career_carry_to_td'] = player_rolling_sort['cusum_ra'] / player_rolling_sort['tdr']

In [196]:
#Carries to fumbles lost ratio
player_rolling_sort['career_carry_to_fuml'] = player_rolling_sort['cusum_ra'] / player_rolling_sort['cusum_fuml']

#### Recent stats

In [62]:
#Yards per rush
player_rolling_sort['recent_ryds_per_carry'] = player_rolling_sort['recent_ry'] / player_rolling_sort['recent_ra']

In [63]:
#Carries to touchdown ratio
player_rolling_sort['recent_carry_to_td'] = player_rolling_sort['recent_ra'] / player_rolling_sort['recent_tdr']

In [64]:
#Carries to fumbles lost ratio
player_rolling_sort['recent_carry_to_fuml'] = player_rolling_sort['recent_ra'] / player_rolling_sort['recent_fuml']

### Final checks

In [65]:
player_rolling_sort.columns

Index(['pk', 'gid', 'seas', 'wk', 'player', 'team', 'pos1', 'nflid', 'pa',
       'pc', 'py', 'ints', 'tdp', 'ra', 'sra', 'ry', 'tdr', 'fuml', 'trg',
       'rec', 'recy', 'tdrec', 'exp', 'height', 'weight', 'dob', 'dv', 'forty',
       'bench', 'vertical', 'broad', 'shuttle', 'cone', 'arm', 'hand',
       'cusum_pa', 'cusum_pc', 'cusum_py', 'cusum_ints', 'cusum_tdp',
       'cusum_ra', 'cusum_sra', 'cusum_ry', 'cusum_tdr', 'cusum_fuml',
       'cusum_trg', 'cusum_rec', 'cusum_recy', 'cusum_tdrec', 'recent_pa',
       'recent_pc', 'recent_py', 'recent_ints', 'recent_tdp', 'recent_ra',
       'recent_sra', 'recent_ry', 'recent_tdr', 'recent_fuml', 'recent_trg',
       'recent_rec', 'recent_recy', 'recent_tdrec', 'age', 'career_comp_pct',
       'career_td_to_int', 'career_yds_per_comp', 'recent_comp_pct',
       'recent_td_to_int', 'recent_yds_per_comp', 'recent_ryds_per_carry',
       'recent_carry_to_td', 'recent_carry_to_fuml'],
      dtype='object')

In [66]:
player_rolling_sort[player_rolling_sort['gid'].count()]

(94759, 73)

In [77]:
#percentage of players that played in more than 4 games
game_thres = 10
sum(player_rolling_sort.groupby(by='player')['gid'].count() > game_thres) / len(player_rolling_sort.groupby(by='player')['gid'].count())

0.5866666666666667

## Game features

### Read in data

In [198]:
game = pd.read_csv('https://raw.githubusercontent.com/jchristo12/fantasy_football/master/data/predictor/game.csv')

In [202]:
#change the data types to the correct format
game = game.astype({'v': 'category',
                   'h': 'category',
                   'day': 'category',
                   'cond': 'category',
                   'stad': 'category',
                   'wdir': 'category',
                   'surf': 'category'})

### Feature creation

In [212]:
#add underdog category
game['udog'] = pd.Series(np.where(game['sprv'] > 0, game['v'], game['h']))
game = game.astype({'udog': 'category'})

In [214]:
#combine game and offensive player data
test = df_sorted_cusum.join(game, on='gid', how='left', lsuffix='_poff', rsuffix='_game')

In [216]:
test.head()

,pk,gid_poff,seas,wk,player,team,pos1,pa,pc,py,...,stad,temp,wdir,surf,humd,ou,sprv,ptsv,ptsh,udog
74708,3999&AA-0025,3999,2015,1,AA-0025,DET,RB,0,0,0,...,Raymond James Stadium,84.0,NNE,Grass,66.0,41.0,3.0,42,14,TEN
74935,4010&AA-0025,4010,2015,2,AA-0025,DET,RB,0,0,0,...,Soldier Field,70.0,S,Grass,47.0,46.0,-2.0,48,23,CHI
75490,4036&AA-0025,4036,2015,3,AA-0025,DET,RB,0,0,0,...,Lambeau Field,67.0,N,DD GrassMaster,93.0,49.0,6.5,28,38,KC
75814,4052&AA-0025,4052,2015,4,AA-0025,DET,RB,0,0,0,...,NRG Stadium,NaN,NaN,Grass,NaN,46.0,-3.0,27,20,HOU
76021,4062&AA-0025,4062,2015,5,AA-0025,DET,RB,0,0,0,...,AT&T Stadium,NaN,NaN,AstroTurf,NaN,49.5,-8.5,30,6,DAL


This doesn't look right... will have to dig into this more